In [32]:
#Lets import required packages

import pandas as pd
from pandas import Series,DataFrame
import numpy as np

import os

from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import KFold,cross_val_score

In [2]:
#Load the dataset

San_train = pd.read_csv(r'E:\Study\Data Challenges\kaggle\Santander\train\train.csv')
San_test = pd.read_csv(r'E:\Study\Data Challenges\kaggle\Santander\test\test.csv')

In [3]:
#Checking for Categorical and numercial data and seperating the column names

San_categorical = []
San_numerical = []
for col in San_train.columns:
    if San_train[col].dtype == 'object':
        San_categorical.append(col)
    else:
        San_numerical.append(col)

In [4]:
San_categorical

#Results show that complete data set is built of int or float values
#So we dont need to use any dummies here

[]

In [5]:
### looking at the sahpe
San_train.shape

(76020, 371)

In [7]:
#to find columns with unique values

#Seperate our id columns from test to make sure that id columns exist for submission

San_test_id = San_test['ID']

San_Target =  San_train['TARGET']

i=0
for col in San_train.columns:
    if len(np.unique(San_train[col].values))==76020:
        print("Column %s has all unique values so we will drop the column" %(col))
        i=i+1
        San_train = San_train.drop(col,1)
        San_test = San_test.drop(col,1)
print("No of columns with all unique values are %s" %i)

#But we are not going to delet the id column

Column ID has all unique values so we will drop the column
No of columns with all unique values are 1


In [8]:
#to find columns with constant valued volumns and remove them

drop_train_col = []
drop_test_col = []
for col in San_train.columns:
            if len(np.unique(San_train[col].values)) == 1:
                San_train = San_train.drop(col,1)
                San_test = San_test.drop(col,1)
                drop_train_col.append(col)
                drop_test_col.append(col)
                print("Column %s has a constant value" %(col))

Column ind_var2_0 has a constant value
Column ind_var2 has a constant value
Column ind_var27_0 has a constant value
Column ind_var28_0 has a constant value
Column ind_var28 has a constant value
Column ind_var27 has a constant value
Column ind_var41 has a constant value
Column ind_var46_0 has a constant value
Column ind_var46 has a constant value
Column num_var27_0 has a constant value
Column num_var28_0 has a constant value
Column num_var28 has a constant value
Column num_var27 has a constant value
Column num_var41 has a constant value
Column num_var46_0 has a constant value
Column num_var46 has a constant value
Column saldo_var28 has a constant value
Column saldo_var27 has a constant value
Column saldo_var41 has a constant value
Column saldo_var46 has a constant value
Column imp_amort_var18_hace3 has a constant value
Column imp_amort_var34_hace3 has a constant value
Column imp_reemb_var13_hace3 has a constant value
Column imp_reemb_var33_hace3 has a constant value
Column imp_trasp_var

In [9]:
# to check for columns whihc are present in train but not in test
#Before doing so lets seperate our target varibale

Target = San_train['TARGET']

for col in San_train.columns:
    if col not in San_test.columns:
        San_train=San_train.drop(col,1)
        print("%s column is present in train but not present in test" %(col))
        drop_train_col.append(col)     
        
        
        
#Similarly removing variables in test data which are not in train

for col in San_test.columns:
    if col not in San_train.columns:
        San_train=San_test.drop(col,1)
        print("%s column is present in train but not present in test" %(col))
        drop_test_col.append(col)

TARGET column is present in train but not present in test


In [11]:
## Checking if the dataset after manipulation is in good shape or not


print(San_train.shape)
print(San_test.shape)
print(len(San_Target))
print(len(San_test_id))

(76020, 335)
(75818, 335)
76020
75818


In [13]:
# Checking for any missing values

San_train.isnull().sum().sum()

San_test.isnull().sum().sum()

#Output is 0 , so there are no missing values in the dataset

0

In [22]:
# To check if the dataset is balanced

print("There are %s number of 0 in the Target" %sum(San_Target==0))
print("There are %s number of 1 in the Target" %sum(San_Target==1))

# output suggest that the dataset is highly imbalnce
# so we have to balance it before building the models

There are 73012 number of 0 in the Target
There are 3008 number of 1 in the Target


In [23]:
# Lets try to random under sampling

San_train['TARGET']=San_Target ##Adding back the target variable to the train dataset to do under sampling

under_san_train = pd.concat([San_train[San_train['TARGET']==0].sample(n=3008,replace=False),San_train[San_train['TARGET']==1]])

Target_under = under_san_train['TARGET']
under_san_train = under_san_train.drop('TARGET',1)


In [27]:
for n_es in range(500,1000,100):
    for max_f in range(2,5,1):
        rf = RandomForestClassifier(n_estimators = n_es,criterion ='entropy',max_features=max_f)
        print("n_es = %s"%n_es)
        print("max_f = %s"%max_f)
        print(cross_val_score(rf,under_san_train,Target_under,cv=10).mean())

n_es = 500
max_f = 2
0.725066445183
n_es = 500
max_f = 3
0.7272303433
n_es = 500
max_f = 4
0.726564784053
n_es = 600
max_f = 2
0.723070874862
n_es = 600
max_f = 3
0.725894795127
n_es = 600
max_f = 4
0.724569213732
n_es = 700
max_f = 2
0.724069767442
n_es = 700
max_f = 3
0.725233665559
n_es = 700
max_f = 4
0.726557585825
n_es = 800
max_f = 2
0.726065337763
n_es = 800
max_f = 3
0.720911406423
n_es = 800
max_f = 4
0.725063122924
n_es = 900
max_f = 2
0.724735326689
n_es = 900
max_f = 3
0.727395348837
n_es = 900
max_f = 4
0.726562015504


In [30]:
## Based on the best case achieved above we will build the random forest classifier

forest = RandomForestClassifier(n_estimators =900 ,criterion ='entropy',max_features=18)
forest = forest.fit(under_san_train,Target_under)
output_under = forest.predict(San_test)

#Then to save the predicted output

RF_1 = pd.DataFrame({'ID':San_test_id,'TARGET':output_under})

os.chdir(r'E:\Study\Data Challenges\kaggle\Santander') #Changing the Dir to save the ouput file

RF_1.to_csv("RF_1.csv")

In [31]:
RF_1.sum()

ID        5752677905
TARGET         21068
dtype: int64

In [ ]:
# Lets try to random over sampling

over_san_train = pd.concat([San_train[San_train['TARGET']==1].sample(n=73012,replace=True),San_train[San_train['TARGET']==0]])

Target_over = under_san_train['TARGET']
over_san_train = under_san_train.drop('TARGET',1)

### CV scoring

for n_es in range(500,1000,100):
    for max_f in range(2,5,1):
        rf = RandomForestClassifier(n_estimators = n_es,n_jobs =-1,criterion ='entropy',max_features=max_f)
        print("n_es = %s"%n_es)
        print("max_f = %s"%max_f)
        print(cross_val_score(rf,over_san_train,Target_over,cv=10).mean())


In [ ]:
## Based on the best case achieved above we will build the random forest classifier

forest = RandomForestClassifier(n_estimators =900 ,criterion ='entropy',max_features=18)
forest = forest.fit(over_san_train,Target_over)
output_over = forest.predict(San_test)

#Then to save the predicted output

RF_1 = pd.DataFrame({'ID':San_test_id,'TARGET':output_under})

os.chdir(r'E:\Study\Data Challenges\kaggle\Santander') #Changing the Dir to save the ouput file

RF_1.to_csv("RF_1.csv")